<a href="https://colab.research.google.com/github/chacha86/pythonai/blob/main/%EC%B6%94%EC%B2%9C%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98_%ED%95%99%EC%8A%B52.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pyspark

df2 = spark.read.option("sep", ",").csv("/test/music/my_data.csv/").toDF('userid', 'artistid', 'playcount')


In [ ]:
%pyspark

df3 = df2.select(df2.userid.cast('int'), df2.artistid.cast('int'), df2.playcount.cast('int'))

In [ ]:
%pyspark
df4 = df3.filter(df3.userid.isNotNull() & df3.artistid.isNotNull() & df3.playcount.isNotNull())

df4

In [ ]:
%pyspark


from pyspark.ml.recommendation import ALS

als = ALS(maxIter=5, regParam=0.01, userCol="userid", itemCol="artistid", ratingCol="playcount",
          coldStartStrategy="drop", rank=2, implicitPrefs=True)
model = als.fit(df4)


In [ ]:
%pyspark

model.write().save('/test/model2')

In [ ]:
%pyspark

from pyspark.ml.recommendation import ALSModel
als_model = ALSModel.load('/test/model2')

In [ ]:
%pyspark

userRecs = model.recommendForAllUsers(3)


userRecs

In [ ]:
%pyspark

userRecs.show(truncate=False)


In [ ]:
%pyspark
from pyspark.sql.functions import *

userRecs.select(userRecs.userid, explode(userRecs.recommendations)).show()

In [ ]:
%pyspark

tmp = userRecs.select(userRecs.userid, explode(userRecs.recommendations))

tmp


In [ ]:
%pyspark

result = tmp.select(tmp.userid, tmp.col.artistid, tmp.col.rating)
result.show()